# COVIDvu

COVID-19 view various charts.

In [ ]:
from utils import autoReloadCode; autoReloadCode()

---
## Pull latest datasets from the JH CSSE GitHub repository

In [ ]:
%cd COVID-19

In [ ]:
%sx git fetch

In [ ]:
%sx git pull

In [ ]:
%cd ..

## Group confirmed cases by country

In [ ]:
import os

import pandas as pd
import numpy as np
import utils

COVID19_PATH=os.path.join(os.path.join(os.getcwd(), 'COVID-19'), 'csse_covid_19_data/csse_covid_19_time_series')
CONFIRMED_FILE=os.path.join(COVID19_PATH, 'time_series_19-covid-Confirmed.csv')

CONFIRMED_FILE

In [ ]:
DEATHS_FILE=os.path.join(COVID19_PATH, 'time_series_19-covid-Deaths.csv')
DEATHS_FILE

In [ ]:
RECOVERY_FILE=os.path.join(COVID19_PATH, 'time_series_19-covid-Recovered.csv')
RECOVERY_FILE

In [ ]:
confirmedCases = pd.read_csv(CONFIRMED_FILE).groupby(['Country/Region']).sum().T

confirmedCases.head()

In [ ]:
confirmedDeaths = pd.read_csv(DEATHS_FILE).groupby(['Country/Region']).sum().T
confirmedDeaths.head()

In [ ]:
confirmedRecovered = pd.read_csv(RECOVERY_FILE).groupby(['Country/Region']).sum().T

confirmedRecovered.head()

In [ ]:
confirmedCasesTrim = confirmedCases.iloc[2:]
confirmedCasesTrim.index = pd.to_datetime(confirmedCasesTrim.index)
confirmedCasesTrim = utils.computeGlobal(confirmedCasesTrim)
confirmedCasesTrim = utils.computeCasesOutside(confirmedCasesTrim, 
                                               ['Mainland China', '!Global'],
                                               '!Outside Mainland China'
                                              )

In [ ]:
confirmedDeathsTrim = confirmedDeaths.iloc[2:]
confirmedDeathsTrim.index = pd.to_datetime(confirmedDeathsTrim.index)
confirmedDeathsTrim = utils.computeGlobal(confirmedDeathsTrim)
confirmedDeathsTrim = utils.computeCasesOutside(confirmedDeathsTrim, 
                                               ['Mainland China', '!Global'],
                                               '!Outside Mainland China'
                                              )

In [ ]:
confirmedRecoveredTrim = confirmedRecovered.iloc[2:]
confirmedRecoveredTrim.index = pd.to_datetime(confirmedRecoveredTrim.index)
confirmedRecoveredTrim = utils.computeGlobal(confirmedRecoveredTrim)
confirmedRecoveredTrim = utils.computeCasesOutside(confirmedRecoveredTrim, 
                                               ['Mainland China', '!Global'],
                                               '!Outside Mainland China'
                                              )

In [ ]:
unknownOutcomes = confirmedCasesTrim - confirmedDeathsTrim - confirmedRecoveredTrim

In [ ]:
mortalityRate = confirmedDeathsTrim/confirmedCasesTrim

In [ ]:
recoveryRate = confirmedRecoveredTrim/confirmedCasesTrim

In [ ]:
knownOutcomeRate = (confirmedCasesTrim - unknownOutcomes)/confirmedCasesTrim

## Plot confirmed by date, filtered by country 

In [ ]:
import visualize

In [ ]:
from ipywidgets import widgets, interact, fixed, interact_manual

countries = list(confirmedCasesTrim.columns)

In [ ]:
from ipywidgets import widgets
from ipywidgets import interact
from ipywidgets import fixed
from ipywidgets import interact_manual
from pandas.core.series import Series
from pandas.core.indexes.base import Index
from numpy import ndarray
from typing import Union
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
import cufflinks as cf
import numpy as np
import pandas as pd

cf.go_offline(connected=False)
init_notebook_mode(connected=False)

In [ ]:
colours = ['red', 'blue', 'green', 'orange']
def plotMultipleTimeSeriesDropdown(df,
                                   dropDownValues,
                                   log=False,
                                   **kwargs
                                   ):
    """

    Parameters
    ----------
    df: A dataframe where the index are distinct time series to plot, and names are displayed as buttons.
        Columns are dates or times.
    yLabel: Axis label of the y-axis
    titleLeft: The string to the left of the name of the time series in the title
    titleRight: The string to the right of the name of the time series in the title
    color: Color of the time series

    Returns
    -------
    A plotly plot where buttons are rows of df, and each row corresponds to a time series.
    """

    #     if name_2:
    #         name = name_2

    ylabel = kwargs.get("y_label", "Total daily count")
    color = kwargs.get("colour", "red")

    # filter out name we want
    try:
        df = df[list(dropDownValues)]
    except KeyError:
        print(f"No countries {dropDownValues}. Did you know mitochondria have their own DNA?")
        return

    fig = go.Figure(layout={"yaxis_title": ylabel,
                            "legend": {"itemsizing": "constant"},
                            "hovermode": "x",
                            #"template": "plotly_dark"
                            })

    for ii, country_name in enumerate(dropDownValues):
        df_this_country = df[country_name]

        xvals = df_this_country.index
        yvals = df_this_country.values

        if log:
            yvals = np.log10(yvals + 1)
            ylabel = "log 10 " + ylabel

        fig.add_trace(
            go.Scatter(x=xvals,
                       y=yvals,
                       name=country_name,
                       line=dict(color=colours[ii]),
                       showlegend=True,
                       mode="lines+markers",
                       marker={"size": 5})
        )

    fig.update_layout(title_text=ylabel)

    return fig

In [ ]:

multiCountry = widgets.SelectMultiple(
    options=countries,
    value=['!Global'],
    #rows=10,
    description='Country',
    disabled=False
)



In [ ]:
interact(plotMultipleTimeSeriesDropdown, 
         df=fixed(confirmedCasesTrim), 
         dropDownValues=multiCountry, 
         #name_2=widgets.Text(value='', placeholder='<enter name of country>', description='Country:',), 
         log=widgets.Checkbox(value=False, description='Log scale')
);

In [ ]:
import plotly.io as pio 
list(pio.templates)

In [ ]:
visualize.plotMultipleTimeSeriesDropdown(confirmedCasesTrim,
                                         yLabel='Total daily count',
                                         color='#690415',
                                         titleLeft='COVID-19 confirmed cases in ',
                                         )                               

In [ ]:
from pandas.core.series import Series
from pandas.core.indexes.base import Index
from numpy import ndarray
from typing import Union
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
import cufflinks as cf

cf.go_offline(connected=False)
init_notebook_mode(connected=False)

colours = ["red", "green", "blue", "yellow", "lightgreen", "indianred", "papayawhip"]*10

def plotMultipleTimeSeriesDropdown(df,
                                   name,
                                   name_2,
                                   log=False,
                                    **kwargs
                                   ):
    """

    Parameters
    ----------
    df: A dataframe where the index are distinct time series to plot, and names are displayed as buttons.
        Columns are dates or times.
    yLabel: Axis label of the y-axis
    titleLeft: The string to the left of the name of the time series in the title
    titleRight: The string to the right of the name of the time series in the title
    color: Color of the time series

    Returns
    -------
    A plotly plot where buttons are rows of df, and each row corresponds to a time series.
    """
        
    if name_2:
        name = name_2
    
    ylabel = kwargs.get("y_label", "Total daily count")
    color = kwargs.get("colour", "red")        


    # filter out name we want
    try:
        df = df[list(name)]
    except KeyError:
        print(f"No countries {name}. Did you know mitochondria have their own DNA?")
        return
    
    if len(df) == 0:
        print(f"No data for countries {name}. Did you know mitochondria have their own DNA?")
        return        
    
    
    fig = go.Figure(layout={"yaxis_title": ylabel,
                            "legend": {"itemsizing": "constant"},
                            "hovermode": "x", 
                            "template": "plotly_dark"
                            })
    
    for ii, country_name in enumerate(name):
        df_this_country = df[country_name]
        
        xvals = df_this_country.index
        yvals = df_this_country.values
        
        if log:
            yvals = np.log10(yvals + 1)
            ylabel = "log 10 " + ylabel
        
        fig.add_trace(
            go.Scatter(x=xvals,
                       y=yvals,
                       name=country_name,
                       line=dict(color=colours[ii]), 
                       showlegend=True, 
                       mode="lines+markers", 
                       marker={"size": 5})
        )

    fig.update_layout(title_text=ylabel)

    return fig

In [ ]:
visualize.plotMultipleTimeSeriesDropdown(confirmedCasesTrim.diff(axis=1),
                                         yLabel='Number of new cases (per day)',
                                         color='#690415',
                                         titleLeft='COVID-19 confirmed new cases ',
                                         titleRight=' (per day)'
                                         )
                               


In [ ]:
visualize.plotMultipleTimeSeriesDropdown(np.log10(confirmedCasesTrim+1),
                                         yLabel='Log10 Total daily count',
                                         color='#690415',
                                         titleLeft='COVID-19 confirmed cases in ',
                                         )
                               


## Confirmed deaths

In [ ]:
visualize.plotMultipleTimeSeriesDropdown(confirmedDeathsTrim,
                                         yLabel='Total daily count',
                                         color='#690415',
                                         titleLeft='COVID-19 confirmed deaths in ',
                                         )

In [ ]:
visualize.plotMultipleTimeSeriesDropdown(confirmedDeathsTrim.diff(axis=1),
                                         yLabel='Number of deaths (per day)',
                                         color='#690415',
                                         titleLeft='COVID-19 confirmed deaths in ',
                                         titleRight=' (per day)'
                                         )

## Untracked cases

In [ ]:
visualize.plotMultipleTimeSeriesDropdown(unknownOutcomes,
                                         yLabel='Total unknown outcomes',
                                         color='#690415',
                                         titleLeft='COVID-19 unknown outcomes from confirmed cases ',
                                         )

## Plot mortality rate

In [ ]:
visualize.plotMultipleTimeSeriesDropdown(mortalityRate,
                                         yLabel='Mortality rate (deaths/cases)',
                                         color='#690415',
                                         titleLeft='COVID-19 mortality rate in ',
                                         )

## Plot recovery rate

In [ ]:
visualize.plotMultipleTimeSeriesDropdown(recoveryRate,
                                         yLabel='Recovery rate (recovered/cases)',
                                         color='#690415',
                                         titleLeft='COVID-19 recovery rate in ',
                                         )

## Plot reporting rate

In [ ]:
knownOutcomeRate
visualize.plotMultipleTimeSeriesDropdown(knownOutcomeRate,
                                         yLabel='Known outcome rate',
                                         color='#690415',
                                         titleLeft='COVID-19 known outcome rate in ',
                                         )

In [ ]:

def show_table(data_frame, num):
    
    num = min(num, len(data_frame))
    
    return data_frame.iloc[:num]

interact(show_table, data_frame=fixed(confirmedCasesTrim), num=widgets.IntSlider(min=5, max=50, value=10, step=2))